In [5]:
import cv2
import numpy as np

In [13]:
cap = cv2.VideoCapture('sample 30 frame 1 min.mp4')
if (cap.isOpened()== False): 
    print("Error opening video stream or file")

gray_frames = []    
while(cap.isOpened()):
  
    ret, frame = cap.read()
    if ret == True:
        gray = cv2.cvtColor(frame, cv.COLOR_BGR2GRAY)
        gray_frames.append(gray)
        cv2.imshow('Frame', gray)

        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else: 
        break

cap.release()
cv2.destroyAllWindows()

# Resources & References

1. https://learnopencv.com/read-write-and-display-a-video-using-opencv-cpp-python/
2. https://docs.opencv.org/master/dd/d43/tutorial_py_video_display.html